# Feature Engineering and Model selection

## Setup

In [211]:
import pandas as pd
import numpy as np

In [212]:
df = pd.read_csv("../data/housing_data.csv", index_col="id")

# Missing neighborhood set to NaN
df.neighborhood = df.neighborhood.apply(lambda x: np.nan if x == "BA" else x)

# Drop null prices and outliers
df = df[~(df.prices.isna())]
df = df[~(df.areas > 200)]
df = df[~(df.bedrooms > 5)]
df = df[~(df.bathrooms > 5)]
df = df[~(df.parkingSpots > 5)]
df = df[~(df.prices > 1700000)]

### Our data

In [213]:
df.head()

,areas,bedrooms,bathrooms,parkingSpots,prices,type,neighborhood
id,,,,,,,
2667338201,51.0,2.0,1.5,1.0,295000.0,NaN,Santa Teresa
2643720430,94.0,3.0,4.0,2.0,629000.0,apartment,Cidade Jardim
2681509397,66.0,2.0,2.0,2.0,460000.0,apartment,Imbuí
2671493238,82.0,3.0,3.0,2.0,649990.0,apartment,Piatã
2592933397,50.0,2.0,1.0,1.0,184900.0,apartment,Luís Anselmo


In [214]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2670 entries, 2667338201 to 2648775965
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   areas         2670 non-null   float64
 1   bedrooms      2665 non-null   float64
 2   bathrooms     2667 non-null   float64
 3   parkingSpots  2137 non-null   float64
 4   prices        2670 non-null   float64
 5   type          2659 non-null   object 
 6   neighborhood  2636 non-null   object 
dtypes: float64(5), object(2)
memory usage: 166.9+ KB


## Feature Engineering

### Neighborhood Price/Area

Since this relationship behaves differently for apartments/houses. We'll have different values for each. 

In [215]:
neighborhood_house_price_sum =  df[df.type == "house"].groupby("neighborhood").prices.sum()
neighborhood_house_area_sum =  df[df.type == "house"].groupby("neighborhood").areas.sum()

neighborhood_apartment_price_sum =  df[df.type == "apartment"].groupby("neighborhood").prices.sum()
neighborhood_apartment_area_sum =  df[df.type == "apartment"].groupby("neighborhood").areas.sum()


neighborhoods_area_price = pd.DataFrame({
    "neighborhood_apartment_area_price": neighborhood_apartment_price_sum / neighborhood_apartment_area_sum,
    "neighborhood_house_area_price": neighborhood_house_price_sum / neighborhood_house_area_sum
    })

In [216]:
def set_neighborhood_area_price(x):
    if x.type == "house" and x.neighborhood in neighborhoods_area_price.index:
        return neighborhoods_area_price.loc[x.neighborhood]["neighborhood_house_area_price"]
    elif x.type == "apartment" and x.neighborhood in neighborhoods_area_price.index:
        return neighborhoods_area_price.loc[x.neighborhood]["neighborhood_apartment_area_price"]
    else: return np.nan

df["neighborhood_area_price"] = df.apply(set_neighborhood_area_price, axis=1)
df.head()

,areas,bedrooms,bathrooms,parkingSpots,prices,type,neighborhood,neighborhood_area_price
id,,,,,,,,
2667338201,51.0,2.0,1.5,1.0,295000.0,NaN,Santa Teresa,NaN
2643720430,94.0,3.0,4.0,2.0,629000.0,apartment,Cidade Jardim,6641.723356
2681509397,66.0,2.0,2.0,2.0,460000.0,apartment,Imbuí,6236.615487
2671493238,82.0,3.0,3.0,2.0,649990.0,apartment,Piatã,6219.599109
2592933397,50.0,2.0,1.0,1.0,184900.0,apartment,Luís Anselmo,3859.821429


## Model Evaluation

### Prepare training data

In [217]:
df.type = df.type.replace({"house": 0, "apartment": 1})

X = df.drop("prices", axis=1)
y = df.prices

In [218]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

from sklearn.model_selection import GridSearchCV

### Set Pipeline function

In [219]:
def create_pipeline(model, training_data):
    numerical_cols = training_data.select_dtypes(include='number').columns
    categorical_cols = training_data.select_dtypes(include='object').columns

    # Preprocessing for numerical data
    numerical_transformer = SimpleImputer(strategy='median')

    # Preprocessing for categorical data
    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ])

    #Bundling
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numerical_transformer, numerical_cols),
            ('cat', categorical_transformer, categorical_cols)
        ])

    return Pipeline(steps=[('preprocessor', preprocessor), ('model', model)])

### Random Forest Regressor

In [220]:
from sklearn.ensemble import RandomForestRegressor

pipe = create_pipeline(RandomForestRegressor(random_state=1, n_jobs=6), X)

#### Grid Search

In [221]:
params = {
    "model__n_estimators" : [100, 250, 300, 500],
    'model__bootstrap': [True, False],
    "model__max_depth" : [2, 5, 10, None]
}

cv = GridSearchCV(pipe, params, cv=5, scoring=["r2", "neg_mean_absolute_error"], refit="r2", verbose=2)

In [222]:
# cv.fit(X, y)

In [223]:
# random_forest_results = pd.DataFrame(cv.cv_results_)
# random_forest_results[["mean_fit_time", "mean_score_time", "param_model__bootstrap", "param_model__n_estimators", "param_model__max_depth",
#           "mean_test_r2", "rank_test_r2", "mean_test_neg_mean_absolute_error", "std_test_neg_mean_absolute_error"]].sort_values("rank_test_r2").head()

### XGBoost

In [224]:
from xgboost import XGBRegressor

pipe = create_pipeline(XGBRegressor(random_state=1, n_jobs=6), X)

#### GridSearchCV

In [225]:
params = {
    "model__n_estimators" : [200, 300, 500],
    "model__learning_rate": [0.05],
    "model__max_depth": [1, 3, 5],
    "model__min_child_weight": [0.01, 1, 3, 5],
    "model__gamma": [0.05]
}

cv = GridSearchCV(pipe, params, cv=5, scoring=["r2", "neg_mean_absolute_error"], refit="r2", verbose=2)

In [226]:
cv.fit(X, y)

Fitting 5 folds for each of 108 candidates, totalling 540 fits
[CV] END model__gamma=0.05, model__learning_rate=0.05, model__max_depth=1, model__min_child_weight=0.01, model__n_estimators=200; total time=   0.0s
[CV] END model__gamma=0.05, model__learning_rate=0.05, model__max_depth=1, model__min_child_weight=0.01, model__n_estimators=200; total time=   0.0s
[CV] END model__gamma=0.05, model__learning_rate=0.05, model__max_depth=1, model__min_child_weight=0.01, model__n_estimators=200; total time=   0.0s


[CV] END model__gamma=0.05, model__learning_rate=0.05, model__max_depth=1, model__min_child_weight=0.01, model__n_estimators=200; total time=   0.0s
[CV] END model__gamma=0.05, model__learning_rate=0.05, model__max_depth=1, model__min_child_weight=0.01, model__n_estimators=200; total time=   0.0s
[CV] END model__gamma=0.05, model__learning_rate=0.05, model__max_depth=1, model__min_child_weight=0.01, model__n_estimators=300; total time=   0.0s
[CV] END model__gamma=0.05, model__learning_rate=0.05, model__max_depth=1, model__min_child_weight=0.01, model__n_estimators=300; total time=   0.0s
[CV] END model__gamma=0.05, model__learning_rate=0.05, model__max_depth=1, model__min_child_weight=0.01, model__n_estimators=300; total time=   0.0s
[CV] END model__gamma=0.05, model__learning_rate=0.05, model__max_depth=1, model__min_child_weight=0.01, model__n_estimators=300; total time=   0.0s
[CV] END model__gamma=0.05, model__learning_rate=0.05, model__max_depth=1, model__min_child_weight=0.01, m

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         SimpleImputer(strategy='median'),
                                                                         Index(['areas', 'bedrooms', 'bathrooms', 'parkingSpots', 'type',
       'neighborhood_area_price'],
      dtype='object')),
                                                                        ('cat',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(strategy='most_frequent')),
                                                                                         ('onehot',
                                                                                          OneHotEncoder(handle_unknown=...
                                                     monotone_constraints=None,
                                                     multi_strategy=None,
                                                     n_estimators=None,
                                                     n_jobs=6,
                                                     num_parallel_tree=None,
                                                     random_state=1, ...))]),
             param_grid={'model__gamma': [0.05, 0.01, 1],
                         'model__learning_rate': [0.05],
                         'model__max_depth': [1, 3, 5],
                         'model__min_child_weight': [0.01, 1, 3, 5],
                         'model__n_estimators': [200, 300, 500]},
             refit='r2', scoring=['r2', 'neg_mean_absolute_error'], verbose=2)

In [227]:
xgboost_results = pd.DataFrame(cv.cv_results_)
xgboost_results[["rank_test_r2", "mean_test_r2", "std_test_r2", "mean_test_neg_mean_absolute_error", "std_test_neg_mean_absolute_error"]].sort_values("rank_test_r2").head(10)

,rank_test_r2,mean_test_r2,std_test_r2,mean_test_neg_mean_absolute_error,std_test_neg_mean_absolute_error
58,1,0.856242,0.043171,-37935.118030,4367.549414
94,1,0.856242,0.043171,-37935.118030,4367.549414
22,1,0.856242,0.043171,-37935.118030,4367.549414
93,4,0.856230,0.045395,-37929.501577,4245.224136
21,4,0.856230,0.045395,-37929.501577,4245.224136
57,4,0.856230,0.045395,-37929.501577,4245.224136
59,7,0.855003,0.042489,-38112.486150,4520.685422
95,7,0.855003,0.042489,-38112.486150,4520.685422
23,7,0.855003,0.042489,-38112.486150,4520.685422
18,10,0.854868,0.045827,-37984.047052,4254.350588


In [230]:
xgboost_results.head(10)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_model__gamma,param_model__learning_rate,param_model__max_depth,param_model__min_child_weight,param_model__n_estimators,params,...,std_test_r2,rank_test_r2,split0_test_neg_mean_absolute_error,split1_test_neg_mean_absolute_error,split2_test_neg_mean_absolute_error,split3_test_neg_mean_absolute_error,split4_test_neg_mean_absolute_error,mean_test_neg_mean_absolute_error,std_test_neg_mean_absolute_error,rank_test_neg_mean_absolute_error
0,0.055512,0.003118,0.007102,0.000666,0.05,0.05,1,0.01,200,"{'model__gamma': 0.05, 'model__learning_rate':...",...,0.054313,97,-45017.181531,-43862.260087,-43706.337262,-48165.019890,-37954.267688,-43741.013291,3307.650757,73
1,0.068080,0.029866,0.006601,0.000491,0.05,0.05,1,0.01,300,"{'model__gamma': 0.05, 'model__learning_rate':...",...,0.054604,79,-45388.092792,-43353.623782,-43386.265808,-48010.228772,-38831.874601,-43794.017151,3010.002782,85
2,0.078310,0.000980,0.006801,0.000400,0.05,0.05,1,0.01,500,"{'model__gamma': 0.05, 'model__learning_rate':...",...,0.054026,91,-46251.141912,-43443.859423,-43425.568275,-48192.702928,-39250.828385,-44112.820185,3025.777618,103
3,0.039004,0.001046,0.006601,0.000800,0.05,0.05,1,1,200,"{'model__gamma': 0.05, 'model__learning_rate':...",...,0.054313,97,-45017.181531,-43862.260087,-43706.337262,-48165.019890,-37954.267688,-43741.013291,3307.650757,73
4,0.057301,0.009543,0.006793,0.001709,0.05,0.05,1,1,300,"{'model__gamma': 0.05, 'model__learning_rate':...",...,0.054604,79,-45388.092792,-43353.623782,-43386.265808,-48010.228772,-38831.874601,-43794.017151,3010.002782,85
5,0.106817,0.018388,0.008200,0.001721,0.05,0.05,1,1,500,"{'model__gamma': 0.05, 'model__learning_rate':...",...,0.054026,91,-46251.141912,-43443.859423,-43425.568275,-48192.702928,-39250.828385,-44112.820185,3025.777618,103
6,0.043108,0.001880,0.013802,0.014135,0.05,0.05,1,3,200,"{'model__gamma': 0.05, 'model__learning_rate':...",...,0.054313,97,-45017.181531,-43862.260087,-43706.337262,-48165.019890,-37954.267688,-43741.013291,3307.650757,73
7,0.067662,0.010720,0.008001,0.001095,0.05,0.05,1,3,300,"{'model__gamma': 0.05, 'model__learning_rate':...",...,0.054873,85,-45312.187185,-43349.163836,-43381.237564,-48026.460809,-38901.127385,-43794.035356,2984.194072,91
8,0.115250,0.039158,0.008401,0.002498,0.05,0.05,1,3,500,"{'model__gamma': 0.05, 'model__learning_rate':...",...,0.054633,76,-46126.094090,-43408.660829,-43476.265793,-48149.130144,-39250.200093,-44082.070190,2996.212881,97
9,0.052308,0.026182,0.006600,0.001356,0.05,0.05,1,5,200,"{'model__gamma': 0.05, 'model__learning_rate':...",...,0.054313,97,-45017.181531,-43862.260087,-43706.337262,-48165.019890,-37954.267688,-43741.013291,3307.650757,73


In [229]:
xgboost_results.loc[0].params

{'model__gamma': 0.05,
 'model__learning_rate': 0.05,
 'model__max_depth': 1,
 'model__min_child_weight': 0.01,
 'model__n_estimators': 200}